**Ładowanie paczek**

In [175]:
from datetime import datetime as dt, date
import json

**Funkcje wspomagające**

Zadanie zostało rozłożone na poszczególne funkcje.

Funkcja szuka indeksy symboli dla wyczyszczenia danych. Te indeksy będą wykorzystane do fragmentacji wiersza danych zgodnie z treścią zadania.

In [176]:
def find_indexes(symbol, input_string): 
    return [i for i in range(len(input_string)) if input_string[i] == symbol] #generator listy

Funkcja wyczyszcza dane, pozbywając się symboli otaczających fragmenty opisu sprawy. Poza tym sprawdza warunki na uszkodzone dane (4 składowych opisu i sparowane znaki). Uszkodzone dane nie zostają zachowywane, bo nie wymaga to treść zadania, jednak  liczba uszkodzonych spraw jest liczona. 

In [177]:
delimiters=['!', '#', '$', '%'] #wejściowe symbole kodujące
def get_clean_data(input_raw_data):
    ok_cases = []
    count_damaged = 0
    for case in input_raw_data: #dla każdej sprawy 
        ok_compose_list = [] #łączenie wyczyszczonych fragmentów danych sprawy
        flag_damaged = True #flaga służy do wskazania czy dane są uszkodzone
        for symbol in delimiters: #sprawdzany jest każdy symbol
            if case.count(symbol) == 2: #niesparowane znaki
                indexes = find_indexes(symbol, case) #funkcja podana wyżej
                ok_compose_list.append(case[(indexes[0]+1):indexes[1]]) #fragment wnętrza bez symboli
            else:
                flag_damaged = False #gdy warunek niesparowania znaków nie jest spełniony
        if len(ok_compose_list) == 4 and flag_damaged != False: #4 składowych opisu i sparowane znaki
            ok_cases.append(ok_compose_list)
        else:
            count_damaged+=1 #liczba spraw uszkodzonych
    return ok_cases, count_damaged

Funkcja polega na selekcji spraw przeterminowanych za pomocą porównania daty raportu i terminu sprawy. Użytkownik zapewnia wprowadzenie daty w formacie ISO). Tworzone zatem są osobne listy dla spraw niezakończonych i przeterminowanych.

In [178]:
def is_overdue(next_step_data, report_date):
    overdue_cases = []
    ok_cases = []
    for case in next_step_data:
        if date.fromisoformat(case[3]) > report_date: #po wyczyszczeniu data znajduje się na czwartym miejscu, format ISO
            ok_cases.append(case)
        else:
            overdue_cases.append(case)
    return ok_cases, overdue_cases

Funkcja przyjmuje kwotę jako wartość tekstową i przekształca ją do kwoty z odsetkami, które liczone są według wzoru
liczba_dni_zaległości\*(r/365)\*kwota_zaległości 

In [179]:
def overdue_calc(text_value, r_day, liczba_dni_zaległości):
    original_value = float(text_value[:-3]) #bez wskazania waluty zł
    overdue_value = round(original_value+liczba_dni_zaległości*r_day*original_value, 2)
    return overdue_value

**Funkcja końcowa Wind**

Funkcja ma przyjąć nazwy dwóch plików, stopę procentową i datę raportu. Pierwszy parametr - nazwa pliku z danymi surowymi w określonym w treści zadania formacie. Drugi - nazwa pliku wyjściowego w formacie .json. Trzeci - roczna stopa procentowa. Czwarty - data raportu, która jest w podana w formacie ISO (więc nie wymagana jest wymuszona konwertacja).
Zadaniem funkcji jest identyfikacja spraw przeterminowanych czyli tych, termin których jest wcześniejszy od daty raportu. 
Funkcja zwraca liczbę spraw przeterminowanych, liczbę spraw uszkodzonych, liczbę spraw niezakończonych. Plik wyściowy zawiera sprawy przeterminowane wraz z kwotami z odsetkami.

In [180]:
def wind(lista_spraw, lista_spraw_przeterm, r=0.55, date=date.today()): 
    if lista_spraw.endswith('.txt') != True or lista_spraw_przeterm.endswith('.json') != True:
        return ('Proszę podać pliki z właściwym rozszerzeniem: .txt dla pliku wejściowego z listą spraw oraz .json dla pliku wyjściowego z listą spraw zaległych')
    date = dt.strptime(str(date), "%Y-%m-%d").date() #zapewnienie wymaganego formatu ISO
    raw_data = open(lista_spraw, 'r', encoding='utf').readlines() #sprawy są oddzielone przez \n
    clean_data, count_damaged = get_clean_data(raw_data) #funkcja podana wyżej
    ok_data, overdue_data = is_overdue(clean_data, date) #funkcja podana wyżej
    clean_overdue_data = []
    for case in overdue_data:
        clean_compose_list = []
        clean_compose_list.append(case[0]) #nr sprawy
        clean_compose_list.append(case[1]) #imię, nazwisko
        clean_compose_list.append(overdue_calc(case[2], r/365, (date-date.fromisoformat(case[3])).days)) #przeskalowanie rocznej stopy na dzienną oraz różnica w dniach
        clean_overdue_data.append(clean_compose_list)
    output_file = open(lista_spraw_przeterm, 'w', encoding='utf')
    json.dump(clean_overdue_data, output_file, ensure_ascii = False) #utf-8
    print(f'Liczba spraw przeterminowanych: {len(overdue_data)}\nLiczba spraw uszkodzonych: {count_damaged}\nLiczba spraw niezakończonych: {len(ok_data)}')
    return len(overdue_data), count_damaged, len(ok_data)

**Przykłady**

In [181]:
wind('windyk2uszk.xlsx', 'windyk2przeterm.xlsx')

'Proszę podać pliki z właściwym rozszerzeniem: .txt dla pliku wejściowego z listą spraw oraz .json dla pliku wyjściowego z listą spraw zaległych'

In [186]:
wind('windyk2uszk.txt', 'windyk2przeterm.json')

Liczba spraw przeterminowanych: 20
Liczba spraw uszkodzonych: 4
Liczba spraw niezakończonych: 6


(20, 4, 6)

In [187]:
wind('windyk5.txt', 'windyk5przeterm.json')

Liczba spraw przeterminowanych: 23
Liczba spraw uszkodzonych: 0
Liczba spraw niezakończonych: 7


(23, 0, 7)

In [184]:
wind('windyk2uszk.txt', 'windyk5przeterm.json', 0.25, '2023-10-01') 

Liczba spraw przeterminowanych: 17
Liczba spraw uszkodzonych: 4
Liczba spraw niezakończonych: 9


(17, 4, 9)